In [22]:
%%time

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from geopy.distance import geodesic

pd.reset_option('display.max_rows',None)

df = pd.read_csv('output.csv')

CPU times: user 1.25 s, sys: 112 ms, total: 1.36 s
Wall time: 1.37 s


In [2]:
# See how many unique values do we have!
for col in list(df.columns):
    print(str(df[col].value_counts().count())+' ' + col)

10298 Unnamed: 0
10298 app_id
9598 fsaccountholdername
119 fsbankid
9950 fsaccountnumber
163 close_date
9351 name
9957 address
5375 aadhaardob
10289 pan
7827 tsadhaarid
9989 tsapplicationcustomerip
2 fpd5
10292 mobilephone
2 is_agr
10286 email
2 isgeofake
1291 geolatitude
6562 geolongitude
10298 dt
9951 social_data_resp
9424 bureau_personal_info
9942 pan_premium_data


In [3]:
df = df.drop(columns=['Unnamed: 0'])
(df.isnull()*1).sum() # See how many NULL values do we have per each column!

app_id                        0
fsaccountholdername           0
fsbankid                      0
fsaccountnumber               0
close_date                 5297
name                          0
address                       1
aadhaardob                    0
pan                           0
tsadhaarid                  132
tsapplicationcustomerip       1
fpd5                          0
mobilephone                   0
is_agr                        0
email                         5
isgeofake                     0
geolatitude                2180
geolongitude               2180
dt                            0
social_data_resp            347
bureau_personal_info        348
pan_premium_data            347
dtype: int64

#### Геолокация IP: Сравните IP-адрес (TSApplicationCustomerIp) с географическим местоположением, указанным пользователем. Если они не совпадают (например, человек заявляет, что он находится в Индии, а IP-адрес принадлежит другой стране), это может быть подозрительным.

In [4]:
def get_ip_geolocation(ip_address, access_token):
    url = f'https://ipinfo.io/{ip_address}/json?token={access_token}'
    response = requests.get(url)
    data = response.json()
    if 'loc' in data and 'country' in data:
        latitude, longitude = map(float, data['loc'].split(','))
        country = data['country']
        return latitude, longitude, country
    return None, None, None

def check_location(ip_address, access_token, declared_latitude, declared_longitude):
    ip_lat, ip_lng, country = get_ip_geolocation(ip_address, access_token)
    
    if ip_lat is None or country is None:
        return False, None, None, None, None

    if country != 'IN':
        return False, None, ip_lat, ip_lng, country

    if pd.isnull(declared_latitude) or pd.isnull(declared_longitude):
        return False, None, ip_lat, ip_lng, country

    distance = geodesic((ip_lat, ip_lng), (declared_latitude, declared_longitude)).kilometers
    return distance < 100, distance, ip_lat, ip_lng, country

def batch_check_locations(dataframe, access_token):
    results = []
    for index, row in dataframe.iterrows():
        ip_address = row['tsapplicationcustomerip']
        declared_latitude = row['geolatitude']
        declared_longitude = row['geolongitude']

        is_match, dist, re_latitude, re_longitude, country = check_location(ip_address, access_token, declared_latitude, declared_longitude)
        
        result = {
            'IP_Address': ip_address,
            'Declared_Latitude': declared_latitude,
            'Real_Latitude': re_latitude,
            'Declared_Longitude': declared_longitude,
            'Real_Longitude': re_longitude,
            'Real_Country': country,
            'Is_Match': is_match,
            'Distance_km': dist if is_match else None
        }
        results.append(result)
    
    return pd.DataFrame(results)

**Чекнуть трешхолд дистнас в check_location**

In [ ]:
%%time

# Пример использования
# Предположим, у вас есть датафрейм df с колонками 'TSApplicationCustomerIp', 'geolatitude', 'geolongitude'
# и вы хотите проверить их соответствие заявленным координатам в Индии.

# Ваш токен для доступа к ipinfo.io API
access_token = 'd9372d4774e12d'

# Пример датафрейма с данными
data = {
    'TSApplicationCustomerIp': ['103.27.9.44', '8.8.8.8'],
    'geolatitude': [28.6139, 37.4219999],
    'geolongitude': [77.2090, -122.0840575]
}
df_test = pd.DataFrame(data)

geo_feat = ['tsapplicationcustomerip', 'geolatitude', 'geolongitude']

# Массовая проверка
ip_loc_df = batch_check_locations(df.loc[~df['geolatitude'].isnull()][geo_feat], access_token)

# Вывод результатов
ip_loc_df

In [ ]:
geo_feat = ['tsapplicationcustomerip', 'geolatitude', 'geolongitude']

In [ ]:
#ip_loc_df.to_csv('geo_data.csv', index=False)

In [5]:
ip_loc_df = pd.read_csv('geo_data.csv')

In [ ]:
ip_loc_df.Is_Match.value_counts()

In [ ]:
ip_loc_df

In [ ]:
df.loc[~df['geolatitude'].isnull()][['app_id', 'fpd5']].reset_index(drop=True)

In [ ]:
ip_loc_df.IP_Address.value_counts()

In [ ]:
ip_loc_df.Real_Country.value_counts()

In [ ]:
ip_loc_df = ip_loc_df.join(df.loc[~df['geolatitude'].isnull()][['app_id', 'fpd5']].reset_index(drop=True))

In [ ]:
print(ip_loc_df.fpd5.value_counts())
print(ip_loc_df.Is_Match.value_counts())
print('Geo-False',ip_loc_df.loc[ip_loc_df['Is_Match']==False].fpd5.value_counts())
print('Geo-True',ip_loc_df.loc[ip_loc_df['Is_Match']==True].fpd5.value_counts())

#### Множественные заявки с одного IP: Проверьте, не поступают ли заявки от разных пользователей с одного IP-адреса. Это может указывать на мошенническую активность.

In [6]:
# Группировка по IP-адресам и подсчет уникальных имен держателей счета
ip_group = df.groupby('tsapplicationcustomerip')['fsaccountholdername'].count().reset_index()

# Переименование столбцов для удобства
ip_group.columns = ['tsapplicationcustomerip', 'userscount']

# Фильтрация IP-адресов с более чем одним уникальным именем держателя счета
suspicious_ips = ip_group[ip_group['userscount'] > 1]

# Вывод результатов
suspicious_ips.sort_values(by='userscount', ascending=False).head(100)

,tsapplicationcustomerip,userscount
8048,2409:4063:4c1a:d805:9bca:a5eb:af03:b25a,23
8071,2409:4063:6d12:1117:7b47:e2ef:4ff3:ae55,17
8037,2409:4063:4304:fab:31ce:6a66:cae1:e0b9,15
8073,2409:4063:6d12:1117:8510:6e6:edd9:d91c,15
8047,2409:4063:4c1a:d805:21bc:b1db:6fd2:bc57,14
...,...,...
4636,152.58.32.135,2
4631,152.58.31.68,2
4622,152.58.31.166,2
4565,152.58.236.230,2


In [7]:
applications   = df.groupby('tsapplicationcustomerip')['fsaccountholdername'].count().reset_index().sort_values(by='fsaccountholdername', ascending=False)
applications_1 = df.groupby('tsapplicationcustomerip')['fpd5'].sum().reset_index().sort_values(by='fpd5', ascending=False)
applications = pd.merge(applications, applications_1, how='inner', on='tsapplicationcustomerip')
applications['ratio'] = pd.Series(applications['fpd5']/applications['fsaccountholdername']).sort_values(ascending=False)
print(pd.Series(applications['fpd5']/applications['fsaccountholdername']).sort_values(ascending=False).value_counts())
applications

0.0    6729
1.0    3238
0.5      22
Name: count, dtype: int64


,tsapplicationcustomerip,fsaccountholdername,fpd5,ratio
0,2409:4063:4c1a:d805:9bca:a5eb:af03:b25a,23,23,1.0
1,2409:4063:6d12:1117:7b47:e2ef:4ff3:ae55,17,17,1.0
2,2409:4063:4304:fab:31ce:6a66:cae1:e0b9,15,15,1.0
3,2409:4063:6d12:1117:8510:6e6:edd9:d91c,15,15,1.0
4,2409:4063:4c1a:d805:21bc:b1db:6fd2:bc57,14,14,1.0
...,...,...,...,...
9984,152.57.21.52,1,0,0.0
9985,152.57.210.14,1,0,0.0
9986,152.57.210.88,1,0,0.0
9987,152.57.211.243,1,0,0.0


In [30]:
applications.loc[applications['fsaccountholdername']==2]['ratio'].value_counts()

ratio
0.0    22
0.5    22
1.0    18
Name: count, dtype: int64

In [31]:
print(applications.fsaccountholdername.value_counts().sort_values(ascending=False).sort_index())
print(applications.fpd5.value_counts().sort_values(ascending=False).sort_index())
print('Variety of expires for 2 applications:',applications.loc[applications['fsaccountholdername']==2]['ratio'].value_counts())

fsaccountholdername
1     9883
2       62
3       15
4        7
5        3
6        3
7        4
8        2
10       3
12       1
14       2
15       2
17       1
23       1
Name: count, dtype: int64
fpd5
0     6729
1     3198
2       18
3       15
4        7
5        3
6        3
7        4
8        2
10       3
12       1
14       2
15       2
17       1
23       1
Name: count, dtype: int64
Variety of expires for 2 applications: ratio
0.0    22
0.5    22
1.0    18
Name: count, dtype: int64


In [32]:
pd.merge(df, suspicious_ips['tsapplicationcustomerip'], how='right', on = 'tsapplicationcustomerip').fpd5.value_counts()

fpd5
1    348
0     66
Name: count, dtype: int64